# Import packages

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

project_folder = '/Users/dusiyi/Documents/Multifirefly-Project'
os.chdir(project_folder)
sys.path.append(os.path.join(project_folder, 'multiff_analysis', 'methods'))

from data_wrangling import specific_utils, process_monkey_information
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization.matplotlib_tools import plot_behaviors_utils
from non_behavioral_analysis.neural_data_analysis.get_neural_data import neural_data_processing
from non_behavioral_analysis.neural_data_analysis.visualize_neural_data import plot_neural_data, plot_modeling_result
from non_behavioral_analysis.neural_data_analysis.model_neural_data import cca_class, pgam_class, neural_data_modeling, reduce_multicollinearity
from non_behavioral_analysis.neural_data_analysis.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class, decode_target_class
from non_behavioral_analysis.neural_data_analysis.planning_neural import planning_neural_class, planning_neural_utils

import sys
import math
import gc
import subprocess
from pathlib import Path
from importlib import reload

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
import torch
from numpy import pi

# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr

# Neuroscience specific imports
import neo
import rcca

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
print("done")

%load_ext autoreload
%autoreload 2

done
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Basic data

In [2]:
## Retrieve monkey data
PLAYER = "monkey"
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330"
data_item = neural_vs_behavioral_class.NeuralVsBehavioralClass(raw_data_folder_path=raw_data_folder_path)
data_item.retrieve_or_make_monkey_data()
data_item.make_or_retrieve_ff_dataframe(exists_ok=True)
data_item.find_patterns()
data_item.make_PlotTrials_args()

monkey_information = data_item.monkey_information
ff_dataframe = data_item.ff_dataframe

ff_life_sorted = data_item.ff_life_sorted
ff_real_position_sorted = data_item.ff_real_position_sorted
ff_believed_position_sorted = data_item.ff_believed_position_sorted
cluster_around_target_indices = data_item.cluster_around_target_indices
ff_caught_T_new = data_item.ff_caught_T_new
caught_ff_num = len(ff_caught_T_new)
ff_flash_sorted = data_item.ff_flash_sorted
ff_flash_end_sorted = data_item.ff_flash_end_sorted
max_point_index = data_item.max_point_index
min_point_index = data_item.min_point_index




data_item.make_or_retrieve_target_clust_last_vis_df()
target_clust_last_vis_df = data_item.target_clust_last_vis_df
target_clust_last_vis_df['ff_index'] = target_clust_last_vis_df['target_index']


PlotTrials_args = (monkey_information, ff_dataframe, ff_life_sorted, ff_real_position_sorted, ff_believed_position_sorted, cluster_around_target_indices, ff_caught_T_new)


plot_polar_args = (monkey_information,
                    ff_dataframe, 
                    ff_life_sorted,
                    ff_real_position_sorted,
                    ff_caught_T_new,
                    ff_flash_sorted,)


trial_total_num = 2
PLAYER = "monkey"

Retrieved monkey_information
The number of points that were removed due to delta_position exceeding the ceiling is 0
Note: ff_caught_T_sorted is replaced with ff_caught_T_new
Retrieved ff_dataframe from all_monkey_data/processed_data/monkey_Bruno/data_0330/ff_dataframe.h5
When take out monkey subset for GUAT, 643 clusters out of 856 are too close to the target or the last target. Those clusters are filtered out.
The number of new trials that are used to separate stop clusters is 1338
Retrieved target_clust_last_vis_df


# retrieve chunks of time of no cluster around target

In [3]:
exists_ok = True
data_item.make_or_retrieve_target_clust_last_vis_df(exists_ok=exists_ok)
data_item.make_or_retrieve_target_last_vis_df(exists_ok=True)

data_item.target_clust_last_vis_df['nearby_vis_ff_indices'] = data_item.target_clust_last_vis_df['nearby_vis_ff_indices'].apply(
    lambda x: [int(i) for i in x.strip('[]').split(',') if i.strip().isdigit()])

data_item.target_clust_last_vis_df['num_nearby_vis_ff'] = data_item.target_clust_last_vis_df['nearby_vis_ff_indices'].apply(lambda x: len(x))

# add ff_caught_time and ff_caught_point_index
data_item.target_clust_last_vis_df['ff_caught_time'] = data_item.ff_caught_T_new[data_item.target_clust_last_vis_df['target_index'].values]
data_item.target_clust_last_vis_df['ff_caught_point_index'] = np.searchsorted(data_item.monkey_information['time'], data_item.target_clust_last_vis_df['ff_caught_time'].values)

Retrieved target_clust_last_vis_df
Retrieved target_last_vis_df


In [4]:
clust_sub = data_item.target_clust_last_vis_df[data_item.target_clust_last_vis_df['num_nearby_vis_ff'] == 1]
# print percentage of clust_sub
print("Percentage of targets not in a visible cluster out of all targets", len(clust_sub) / len(data_item.target_clust_last_vis_df) * 100)

Percentage of targets not in a visible cluster out of all targets 66.4424514200299


In [5]:
# need 'last_vis_point_index' and 'ff_caught_point_index' and the point index in between 

# then get the neural data and behavioral data for those points

# oh btw....should i actually align each section, as if they are trials???
# maybe i can try both that and continuous time... both can shed light on different behavioral variables
# but for aligning trials, it may require alignment or warping since trial durations vary.

# btw, what does it mean stitch data?

# also, what does it look like to use RNN to model it?
# I thought about the paper that Noah presented on


# btw.......IME

# get neural data (as in other notebooks)

In [63]:
data_item = neural_vs_behavioral_class.NeuralVsBehavioralClass(raw_data_folder_path=raw_data_folder_path,
                                                               one_behav_idx_per_bin=True)
data_item.streamline_preparing_neural_and_behavioral_data()

Retrieved monkey_information
The number of points that were removed due to delta_position exceeding the ceiling is 0
Note: ff_caught_T_sorted is replaced with ff_caught_T_new
Retrieved ff_dataframe from all_monkey_data/processed_data/monkey_Bruno/data_0330/ff_dataframe.h5
When take out monkey subset for GUAT, 643 clusters out of 856 are too close to the target or the last target. Those clusters are filtered out.
The number of new trials that are used to separate stop clusters is 1338
Retrieved all_trial_patterns
Retrieved pattern_frequencies
Retrieved all_trial_features
Retrieved feature_statistics
Retrieved scatter_around_target_df
Updated window width (to get convolved data):  1.25


In [45]:
ref_point_mode='distance'
ref_point_value=-150

normalize = False
eliminate_outliers = False
use_curvature_to_ff_center = False
curv_of_traj_mode = 'distance'
window_for_curv_of_traj=[-25, 25]

# experiment

pn = planning_neural_class.PlanningAndNeural(raw_data_folder_path=raw_data_folder_path,
                                             one_behav_idx_per_bin=False)
pn.streamline_organizing_info(ref_point_mode='time after cur ff visible', ref_point_value=0.1, 
                               curv_of_traj_mode=curv_of_traj_mode, window_for_curv_of_traj=window_for_curv_of_traj, truncate_curv_of_traj_by_time_of_capture=True,
                               use_curvature_to_ff_center=use_curvature_to_ff_center,  eliminate_outliers=eliminate_outliers)
pn.retrieve_neural_data()


pn.get_all_planning_info()


Retrieved monkey_information
The number of points that were removed due to delta_position exceeding the ceiling is 0
Note: ff_caught_T_sorted is replaced with ff_caught_T_new
Removed 0 rows out of 768 rows where cur_ff was not visible bbas or nxt_ff was not visible both bbas and bsans
shared_stops_near_ff_df has 768 rows
Retrieving shared_stops_near_ff_df succeeded
Retrieving stop_0_1_window_-50cm_0cm from all_monkey_data/planning/monkey_Bruno/data_0330/diff_in_curv_df/norm_opt_arc/test/stop_0_1_window_-50cm_0cm succeeded
Retrieving Bruno_stop_0_1 from all_monkey_data/planning/monkey_Bruno/data_0330/heading_info_df/norm_opt_arc/test/Bruno_stop_0_1 succeeded
Updated window width (to get convolved data):  1.25


/Users/dusiyi/Documents/Multifirefly-Project/multiff_analysis/methods/non_behavioral_analysis/neural_data_analysis/planning_neural/planning_neural_class.py:34: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  self.bin_info = self.bin_info.merge(self.monkey_information.drop(


,point_index,nxt_ff_angle,nxt_ff_distance,nxt_arc_curv,nxt_opt_arc_curv,nxt_opt_arc_dheading,time_rel_to_stop,traj_curv,cur_ff_angle,cur_ff_distance,cur_arc_curv,cur_opt_arc_curv,cur_opt_arc_dheading,stop_point_index,bin,angle_from_m_before_stop_to_cur_ff,angle_from_m_before_stop_to_nxt_ff,angle_from_cur_ff_landing_to_nxt_ff
0,2383,-0.18217,312.12656,-0.00116,-0.00146,-0.45559,-1.99175,-0.00488,-0.25033,229.94283,-0.00215,-0.00271,-0.61908,2503,158,-0.55642,0.05417,-0.04811
1,2398,0.07231,271.39960,0.00053,0.00013,0.03405,-1.74176,-0.00425,-0.01285,189.45584,-0.00014,-0.00097,-0.18288,2503,159,-0.55642,0.05417,-0.04811
2,2413,0.17947,223.34478,0.00160,0.00101,0.22623,-1.49374,0.00094,0.08205,140.95016,0.00116,0.00094,0.13310,2503,160,-0.55642,0.05417,-0.04811
3,2428,0.06939,176.35630,0.00079,0.00175,0.30516,-1.24496,0.00367,-0.04810,93.53159,-0.00103,-0.00000,-0.00009,2503,161,-0.55642,0.05417,-0.04811
4,2443,0.03882,170.92798,0.00045,0.00148,0.25062,-0.99594,0.00356,-0.08425,88.10515,-0.00191,-0.00000,-0.00009,2503,162,-0.55642,0.05417,-0.04811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4980,218129,-0.33874,159.56026,-0.00417,-0.00342,-0.56089,0.73038,-0.00342,NaN,NaN,NaN,NaN,NaN,218083,14333,-0.52015,-0.34500,-0.53120
4981,218144,-0.12218,120.88841,-0.00202,-0.00407,-0.47910,0.97962,-0.00635,NaN,NaN,NaN,NaN,NaN,218083,14334,-0.52015,-0.34500,-0.53120
4982,218159,0.04436,73.69671,0.00120,0.00000,0.00007,1.22836,-0.00139,NaN,NaN,NaN,NaN,NaN,218083,14335,-0.52015,-0.34500,-0.53120
4983,218174,0.15775,24.44381,0.00143,0.00143,0.03481,1.47735,0.00143,NaN,NaN,NaN,NaN,NaN,218083,14336,-0.52015,-0.34500,-0.53120


## get behavioral data (new version)

In [ ]:
decoder = decode_target_class.DecodeTargetClass(raw_data_folder_path=raw_data_folder_path,
                                                               bin_width=0.02, window_width=0.1)
decoder.streamline_making_behav_data()


In [125]:
decoder.get_behav_data()

Retrieved target_df


In [126]:
decoder.behav_data.columns

Index(['bin', 'monkey_x', 'monkey_y', 'time', 'point_index', 'monkey_angle',
       'monkey_speed', 'monkey_dw', 'monkey_x_smr', 'LDy', 'RDz', 'LDz',
       'monkey_dw_smr', 'monkey_y_smr', 'RDy', 'monkey_speed_smr',
       'gaze_mky_view_x', 'gaze_mky_view_y', 'gaze_mky_view_angle',
       'gaze_world_x', 'gaze_world_y', 'gaze_mky_view_x_l',
       'gaze_mky_view_y_l', 'gaze_mky_view_angle_l', 'gaze_world_x_l',
       'gaze_world_y_l', 'gaze_mky_view_x_r', 'gaze_mky_view_y_r',
       'gaze_mky_view_angle_r', 'gaze_world_x_r', 'gaze_world_y_r',
       'valid_view_point_l', 'valid_view_point_r', 'valid_view_point',
       'eye_world_speed', 'monkey_ddw', 'monkey_ddv', 'monkey_speeddummy',
       'crossing_boundary', 'delta_distance', 'cum_distance',
       'whether_new_distinct_stop', 'stop_id', 'dt', 'turning_right', 'trial',
       'num_alive_ff', 'num_visible_ff', 'min_ff_distance', 'min_abs_ff_angle',
       'min_abs_ff_angle_boundary', 'min_visible_ff_distance',
       'min_abs_vis

In [120]:
decoder.behav_data.columns

Index(['bin', 'monkey_x', 'monkey_y', 'time', 'point_index', 'monkey_angle',
       'monkey_speed', 'monkey_dw', 'monkey_x_smr', 'LDy', 'RDz', 'LDz',
       'monkey_dw_smr', 'monkey_y_smr', 'RDy', 'monkey_speed_smr',
       'gaze_mky_view_x', 'gaze_mky_view_y', 'gaze_mky_view_angle',
       'gaze_world_x', 'gaze_world_y', 'gaze_mky_view_x_l',
       'gaze_mky_view_y_l', 'gaze_mky_view_angle_l', 'gaze_world_x_l',
       'gaze_world_y_l', 'gaze_mky_view_x_r', 'gaze_mky_view_y_r',
       'gaze_mky_view_angle_r', 'gaze_world_x_r', 'gaze_world_y_r',
       'valid_view_point_l', 'valid_view_point_r', 'valid_view_point',
       'eye_world_speed', 'monkey_ddw', 'monkey_ddv', 'monkey_speeddummy',
       'crossing_boundary', 'delta_distance', 'cum_distance',
       'whether_new_distinct_stop', 'stop_id', 'dt', 'turning_right', 'trial',
       'num_alive_ff', 'num_visible_ff', 'min_ff_distance', 'min_abs_ff_angle',
       'min_abs_ff_angle_boundary', 'min_visible_ff_distance',
       'min_abs_vis

In [128]:
decoder.behav_data[other_features]

,bin,LDy,LDz,RDy,RDz,gaze_mky_view_x,gaze_mky_view_y,gaze_world_x,gaze_world_y,monkey_speed,monkey_angle,any_ff_visible,monkey_dw,monkey_ddw,monkey_ddv,target_distance,target_angle,target_last_seen_time,target_last_seen_distance_frozen,target_last_seen_angle_frozen
0,0,18.80072,-2.33398,16.68520,-2.28906,75.51017,235.92759,75.51017,266.72759,0.00000,1.57080,1,-184.07138,0.00000,0.00000,122.41561,-2.18685,100.00000,400.00000,0.00000
1,1,18.77463,-2.33398,16.73303,-2.51001,72.38645,225.35841,-63.65471,-197.17871,0.00000,-1.53228,0,0.00000,92.03569,0.00000,122.41561,0.91623,100.00000,400.00000,0.00000
2,2,18.79202,-2.32666,16.68375,-2.58691,71.47833,222.48808,-62.85779,-194.27553,0.00000,-1.53228,0,0.00000,0.00000,0.00000,122.41561,0.91623,100.00000,400.00000,0.00000
3,3,18.80217,-2.34009,16.78232,-2.55762,71.81999,222.97226,-63.18055,-194.77251,0.00000,-1.53228,1,0.00000,0.00000,0.00000,122.41561,0.91623,100.00000,400.00000,0.00000
4,4,18.78188,-2.27295,16.73303,-2.52832,73.22415,227.68370,-64.40225,-199.53452,0.00000,-1.53228,1,0.00000,0.00000,0.00000,122.41561,0.91623,100.00000,400.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179175,179942,3.14665,-11.78833,1.03983,-9.01147,1.92273,55.44290,-278.52756,302.12250,67.80524,1.03889,0,0.01268,-0.09604,-18.85785,3.20678,-2.87475,0.29879,49.90243,0.03930
179176,179943,3.01184,-11.80420,0.81369,-9.15918,1.73030,54.89810,-278.59087,302.39781,45.15523,1.03888,0,-0.00044,-0.00634,-26.37328,3.93566,-2.92448,0.31540,49.90243,0.03930
179177,179944,2.87268,-11.96289,0.44115,-9.44849,1.44343,53.61113,-279.30162,301.75791,7.50966,1.03888,0,0.00000,0.00000,-7.52801,4.30283,-2.94300,0.34864,49.90243,0.03930
179178,179945,2.55087,-12.09839,0.39621,-9.35938,1.27435,53.63722,-279.43408,301.86612,0.00266,1.03888,0,0.00000,0.00000,-3.75483,4.30281,-2.94301,0.36514,49.90243,0.03930


In [ ]:
# maybe get: cum time since flash & cum distance since flash

In [130]:
[col for col in decoder.behav_data.columns if col not in monkey_features_to_keep]

['monkey_x_smr',
 'monkey_dw_smr',
 'monkey_y_smr',
 'monkey_speed_smr',
 'delta_distance',
 'cum_distance',
 'stop_id',
 'dt',
 'turning_right',
 'trial',
 'num_alive_ff',
 'num_visible_ff',
 'min_ff_distance',
 'min_abs_ff_angle',
 'min_abs_ff_angle_boundary',
 'min_visible_ff_distance',
 'min_abs_visible_ff_angle',
 'min_abs_visible_ff_angle_boundary',
 'catching_ff',
 'target_index',
 'target_x',
 'target_y',
 'target_angle_to_boundary',
 'target_last_seen_distance',
 'target_last_seen_angle',
 'target_last_seen_angle_to_boundary',
 'target_last_seen_angle_to_boundary_frozen',
 'target_has_disappeared_for_last_time_dummy',
 'target_visible_dummy',
 'current_target_caught_time',
 'last_target_caught_time',
 'time_since_last_capture']

In [ ]:
'monkey_x_smr',
 'monkey_dw_smr',
 'monkey_y_smr',
 'monkey_speed_smr',
  'stop_id',
 'dt',

'target_index', 'current_target_caught_time', 'last_target_caught_time'

In [132]:
decoder.behav_data['delta_distance']

0        0.00000
1        0.00000
2        0.00000
3        0.00000
4        0.00000
           ...  
179175   1.12496
179176   0.74998
179177   0.12501
179178   0.00004
179179   0.00000
Name: delta_distance, Length: 179180, dtype: float64

In [129]:
monkey_features_to_keep = [
'bin', 'point_index', # those are just identifiers
'time', 'monkey_x', 'monkey_y', 'monkey_angle',
'monkey_speed', 'monkey_speeddummy', 'monkey_dw', 'monkey_ddw', 'monkey_ddv',
'LDy', 'RDz', 'LDz', 'RDy',
'gaze_mky_view_x', 'gaze_mky_view_y', 'gaze_mky_view_angle',
'gaze_world_x', 'gaze_world_y', 
'gaze_world_x_l', 'gaze_world_y_l', 
'gaze_world_x_r', 'gaze_world_y_r',
'gaze_mky_view_x_l', 'gaze_mky_view_y_l', 'gaze_mky_view_angle_l', 
'gaze_mky_view_x_r', 'gaze_mky_view_y_r', 'gaze_mky_view_angle_r', 
'valid_view_point_l', 'valid_view_point_r', 'valid_view_point', 'eye_world_speed', 
'crossing_boundary', 'whether_new_distinct_stop', 'bin', 'LDy',
'LDz', 'RDy', 'RDz', 'gaze_mky_view_x', 'gaze_mky_view_y',  'gaze_world_x', 'gaze_world_y', 
'monkey_speed', 'monkey_angle','any_ff_visible',
'monkey_dw', 'monkey_ddw', 'monkey_ddv', 
'target_distance', 'target_angle', 'target_last_seen_time', 
'target_last_seen_distance_frozen', 'target_last_seen_angle_frozen'
]

In [127]:
# originally thought I'll just keep those columns from final_behavioral_data
other_features = [
]
       
       
# might want to not drop angle to boundary, especially since it's close to target

## columns from monkey_information to keep

In [ ]:
# shared columns
['bin', 'point_index',
 'LDy', 'LDz', 'RDy', 'RDz', 
'gaze_mky_view_x_l', 'gaze_mky_view_y_l', 'gaze_mky_view_angle_l', 
'gaze_mky_view_x_r', 'gaze_mky_view_y_r', 'gaze_mky_view_angle_r', 
'eye_world_speed', 'valid_view_point_l','valid_view_point_r',
'monkey_speed', 'monkey_angle', 'monkey_dw', 'monkey_ddw', 'monkey_ddv', 
'monkey_speeddummy', 'whether_new_distinct_stop',
'crossing_boundary', 'delta_distance',
  ]

In [ ]:
# columns for continuous time
# obviously, multicollinearity has to be checked before fitting a model
['time', 'monkey_x', 'monkey_y', 'cum_distance'
 'gaze_world_x_l', 'gaze_world_y_l', 'gaze_world_x_r', 'gaze_world_y_r']

In [ ]:
# columns for aligned trials

## other columns

In [ ]:
# now, make sure you have those columns (from target_df)
monkey_features_to_keep = [
'any_ff_visible',
'target_distance', 'target_angle', 'target_last_seen_time', 
'target_last_seen_distance_frozen', 'target_last_seen_angle_frozen'
]

In [139]:
decoder.behav_data.columns

Index(['bin', 'monkey_x', 'monkey_y', 'time', 'point_index', 'monkey_angle',
       'monkey_speed', 'monkey_dw', 'monkey_x_smr', 'LDy', 'RDz', 'LDz',
       'monkey_dw_smr', 'monkey_y_smr', 'RDy', 'monkey_speed_smr',
       'gaze_mky_view_x', 'gaze_mky_view_y', 'gaze_mky_view_angle',
       'gaze_world_x', 'gaze_world_y', 'gaze_mky_view_x_l',
       'gaze_mky_view_y_l', 'gaze_mky_view_angle_l', 'gaze_world_x_l',
       'gaze_world_y_l', 'gaze_mky_view_x_r', 'gaze_mky_view_y_r',
       'gaze_mky_view_angle_r', 'gaze_world_x_r', 'gaze_world_y_r',
       'valid_view_point_l', 'valid_view_point_r', 'valid_view_point',
       'eye_world_speed', 'monkey_ddw', 'monkey_ddv', 'monkey_speeddummy',
       'crossing_boundary', 'delta_distance', 'cum_distance',
       'whether_new_distinct_stop', 'stop_id', 'dt', 'turning_right', 'trial',
       'num_alive_ff', 'num_visible_ff', 'min_ff_distance', 'min_abs_ff_angle',
       'min_abs_ff_angle_boundary', 'min_visible_ff_distance',
       'min_abs_vis

In [ ]:
# also check columns from _add_ff_info_to_binned_features (a.k.a. which are useful)

In [ ]:
# after all this, check the extra columns not included 

## need to add all_planning_info

In [ ]:
# can benefit from all_planning_info:
'time_rel_to_stop',
'traj_curv', 
'cur_opt_arc_dheading', # turn N.A. to 90 degrees

       
 # let's not include curv info so that we don't get NA; curv is angle/distance so it's harder to get rid of N/A than d-heading


In [140]:
pn.all_planning_info.columns

Index(['point_index', 'nxt_ff_angle', 'nxt_ff_distance', 'nxt_arc_curv',
       'nxt_opt_arc_curv', 'nxt_opt_arc_dheading', 'time_rel_to_stop',
       'traj_curv', 'cur_ff_angle', 'cur_ff_distance', 'cur_arc_curv',
       'cur_opt_arc_curv', 'cur_opt_arc_dheading', 'stop_point_index', 'bin',
       'angle_from_m_before_stop_to_cur_ff',
       'angle_from_m_before_stop_to_nxt_ff',
       'angle_from_cur_ff_landing_to_nxt_ff'],
      dtype='object')

## take out data to be used

In [ ]:
# get valid intervals in behavioral data
def _get_index_of_bins_in_valid_intervals(self, gap_too_large_threshold=100, min_combined_valid_interval_length=50):
    """
    Calculate the midpoints of the time bins and get the indices of bins that fall within valid intervals.
    """
    # Calculate the midpoints of the time bins
    mid_bin_time = (self.time_bins[1:] + self.time_bins[:-1]) / 2

    self.valid_intervals_df = specific_utils.take_out_valid_intervals_based_on_ff_caught_time(
        self.ff_caught_T_new, gap_too_large_threshold=gap_too_large_threshold,
        min_combined_valid_interval_length=min_combined_valid_interval_length
    )

    # Get the indices of bins that fall within valid intervals
    self.valid_bin_mid_time, self.valid_bin_index = general_utils.take_out_data_points_in_valid_intervals(
        mid_bin_time, self.valid_intervals_df
    )

    # # print the number of bins out of total numbers that are in valid intervals
    # print(f"Number of bins in valid intervals based on ff caught time: {len(self.valid_bin_index)} out of {len(mid_bin_time)}"
    #       f" ({len(self.valid_bin_index)/len(mid_bin_time)*100:.2f}%)")


# only take out neural data that have behavioral data
def _match_binned_spikes_to_range_of_behavioral_data(self):
    self.max_bin = self.final_behavioral_data['bin'].max()
    self.binned_spikes_matrix, self.binned_spikes_df = neural_data_processing.prepare_binned_spikes_matrix_and_df(
        self.all_binned_spikes, self.max_bin)
    
    
_get_index_of_bins_in_valid_intervals(self, gap_too_large_threshold=100, min_combined_valid_interval_length=50):